<a href="https://colab.research.google.com/github/Henrymnt/IdentificationofKeyModeStyle/blob/main/IdentificationofKeyModeStyle_Phase3_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Import Dependencies/Libraries
!pip3 install mido
import mido, collections
import math, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import pickle

In [ ]:
#@title Reading Dataset
!git clone https://github.com/Henrymnt/sf_data_2025
piece_data = pd.read_csv("/content/sf_data_2025/piece_data_full.csv")
labels = ['A','Bb','B','C','C#','D','Eb','E','F','F#','G','Ab']

In [ ]:
#@title Data Augmentation (Key)
i=0
piece_data_addition=[]
shuffled_data = piece_data.sample(frac=1)
for frequency in piece_data.key.value_counts():
  current_key = piece_data.key.value_counts().index[i]
  current_frequency=frequency
  while current_frequency <= 800:
    for j in range(0,len(shuffled_data)):
      current_row = shuffled_data.iloc[j]
      # print(current_row)

      if current_row["key"] == current_key:
        print(current_key, current_frequency)
        new_row = current_row.copy()

        keys = list(new_row.keys())
        for k in range(0,len(new_row)):
          if keys[k] != "name" and keys[k] != "path" and keys[k] != "key" and keys[k] != "mode" and keys[k] != "style":
              new_row[keys[k]] += np.random.normal(0, (0.1*new_row[keys[k]]))

        new_row["name"]=new_row["name"] + " (Augmented)"
        new_row["path"]="N/A (This piece of data was augmented)"
        # new_row = new_row.transpose()
        piece_data_addition.append(new_row)
        # print(piece_data_addition)
        current_frequency+=1
      if current_frequency > 800:
        break

  i+=1
piece_data_addition = pd.DataFrame(piece_data_addition)
piece_data.key.value_counts().plot(kind = 'bar')
piece_data_key = pd.concat([piece_data, piece_data_addition], ignore_index=True)
piece_data_key.key.value_counts().plot(kind = 'bar')
# piece_data.describe()
# style_data = pd.Series(piece_data["style"]).value_counts()
# piece_data.key.value_counts().plot(kind = 'bar')
# print(piece_data.key.value_counts().index)

In [ ]:
#@title Preparing Data
X = piece_data_key.drop(['key', 'style', 'mode', 'name', 'path', 'binaryrhythmrate','tripletenterrate','tripletexitrate','quintupletenterrate','quintupletexitrate'], axis = 1)
# x = piece_data.iloc[:, 86:98]
y = piece_data_key["key"]
# scaler = sk.preprocessing.StandardScaler()
# x = scaler.fit_transform(X)
# x = pd.DataFrame(x, columns= X.columns)
# y = scaler.fit_transform(Y)
# y = pd.DataFrame(y, columns= Y.columns)

print("Shape:", X.shape)
print("Shape:", y.shape)

In [ ]:
#@title Training the Model with Linear Regression Classifiers (Key)
r = random.randint(1,16777216)
print(r)
X_train_data_key, X_test_data_key, y_train_data_key, y_test_data_key = sk.model_selection.train_test_split(X,y,test_size=0.1, random_state = r)

key_model = sk.linear_model.LogisticRegression(penalty='l2',
                                           dual=False,
                                           tol=0.0001,
                                           C=1,
                                           fit_intercept=True,
                                           intercept_scaling=1,
                                           class_weight=None,
                                           random_state=None,
                                           solver='lbfgs',
                                           max_iter=100,
                                           verbose=1,
                                           warm_start=True,
                                           n_jobs=None,
                                           l1_ratio=None
                                           )
key_model.fit(X_train_data_key, y_train_data_key)
predictions = key_model.predict(X_test_data_key)
# sk.metrics.confusion_matrix(y_test_data_key, predictions)
ds = sk.metrics.ConfusionMatrixDisplay.from_estimator(key_model, X_test_data_key, y_test_data_key, labels=labels)
ds.ax_.set_title("Confusion Matrix for Key Model")
ds.ax_.set_xlabel("Predicted Key")
ds.ax_.set_ylabel("True Key")
print(sk.metrics.accuracy_score(y_test_data_key, predictions))
print(sk.metrics.balanced_accuracy_score(y_test_data_key, predictions))